### Padding
+ 卷积
    + same padding
        + 大小不变
    + valid padding
        + 变小
+ 池化
    + same padding
        + 变大, 平面外补0
    + valid padding
        + 大小不变

In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

/opt/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/opt/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [ ]:
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)
batch_size = 100
n_batch = mnist.train.num_examples // batch_size

# 初始化值
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1) # 生成一个截断的正太分布
    return tf.Variable(initial)

# 初始化偏置
def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

# 卷积层
def conv2d(x,W):
    # x : input tensor of shape [batch, in_height, in_width, in_channels]
    # W : filter/kernel tensor of shape [filter_height, filetr_width, in_channel, out_channels]
    # strides : strides[0]=strides[3]=1, strides[1] x 方向步长, strides[2] y 方向步长 
    # padding : string from 'SAME', 'VALID'
    return tf.nn.conv2d(x,W,strides=[1,1,1,1], padding='SAME')

# 池化层
def max_pool_2x2(x):
    # ksize [1,x,y,1]
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

# 定义两个placeholder
x = tf.placeholder(tf.float32, [None,784])
y = tf.placeholder(tf.float32, [None,10])

# 改变x的格式转为4D向量[batch,in_height, in_weight, in_channels]
x_image = tf.reshape(x, [-1, 28 , 28, 1])
# 初始化第一个卷积层的权值和偏置
W_conv1 = weight_variable([5,5,1,32]) # 5x5窗口, 32个卷积核从一个平面抽取特征
b_conv1 = bias_variable([32]) # 每一个卷积核一个偏置值

# 把x_image和权值向量进行卷积, 再加上偏置值, 然后应用relu激活函数
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1)+b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

# 初始化第二个卷积层的权重和偏置
W_conv2 = weight_variable([5,5,32,64]) # 5x5窗口, 32个卷积核从一个平面抽取特征
b_conv2 = bias_variable([64]) # 每一个卷积核一个偏置值

# 输入是32个特征图
# 把h_pool1和权值向量进行卷积, 再加上偏置值, 然后应用relu激活函数
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2)+b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

# 把池化层2的输出扁平化为1维度
h_pool2_flat = tf.reshape(h_pool2, [-1,7*7*64])

# 原始图片28x28, 第一次卷积后还是28x28, 第一次池化后变14x14
# 第二次卷积后14x14, 第二次池化后 7x7
# 经过上述操作后得到64张 7x7 的特征图

# 初始化第一个全连接层的权值
W_fc1 = weight_variable([7*7*64, 1024]) # 上一层7*7*24 个神经元, 全连接层1024个神经元
b_fc1 = bias_variable([1024]) # 1024 个神经元

# 求第一个全链接层的输出
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat,W_fc1) + b_fc1)

# keep_prob 
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

# 初始化 第二个全连接层
W_fc2 = weight_variable([1024, 10]) # 上一层7*7*24 个神经元, 全连接层1024个神经元
b_fc2 = bias_variable([10]) # 1024 个神经元

# 计算输出
predict = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

print(predict.shape)
# loss
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=predict))
# optimizer Adam
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
# 把结果放到一个bool表中
correct_prediction = tf.equal(tf.argmax(predict,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(21):
        for batch in range(n_batch):
            batch_xs,batch_ys = mnist.train.next_batch(batch_size)
            sess.run(train_step,feed_dict={x:batch_xs, y:batch_ys, keep_prob:0.7})
        acc = sess.run(accuracy, feed_dict={x:mnist.test.images, y:mnist.test.labels, keep_prob:1.0})
        print("Iter " + str(epoch) + "\t Testing Accuracy = " + str(acc))



Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
(?, 10)
Iter 0	 Testing Accuracy = 0.9519
Iter 1	 Testing Accuracy = 0.9695
Iter 2	 Testing Accuracy = 0.9768
Iter 3	 Testing Accuracy = 0.9788
Iter 4	 Testing Accuracy = 0.9832
Iter 5	 Testing Accuracy = 0.9835
Iter 6	 Testing Accuracy = 0.9861
Iter 7	 Testing Accuracy = 0.9868
Iter 8	 Testing Accuracy = 0.9884
Iter 9	 Testing Accuracy = 0.9886
Iter 10	 Testing Accuracy = 0.9893
Iter 11	 Testing Accuracy = 0.9898
